In [1]:
import sys
if "../../" not in sys.path:
    sys.path.append("../../")

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

import pdaactconn as pc
from trialexplorer import AACTStudySet

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# loading all interventional studies
conn = pc.AACTConnection(source=pc.AACTConnection.REMOTE)
ss = AACTStudySet.AACTStudySet(conn=conn, 
                               tqdm_handler=tqdm)
ss.add_constraint("study_type = 'Interventional'")
ss.load_studies()

262772 studies loaded!


In [4]:
# adding and loading dimensional data
ss.add_dimensions(['sponsors',
                   'result_groups'])
ss.refresh_dim_data()

Successfuly added these 2 dimensions: ['sponsors', 'result_groups']
Failed to add these 0 dimensions: []


Syncing the temp table temp_cur_studies in 526 chunks x 500 records each

Creating index on the temp table
 - Loading dimension sponsors
 -- Loading raw data
 -- Sorting index
 - Loading dimension result_groups
 -- Loading raw data
 -- Sorting index


In [5]:
rg = ss.dimensions['result_groups'].data
sp = ss.dimensions['sponsors'].data

## refreshing so that sponsors only include result-containing studies

In [6]:
res_idx = rg.index.get_level_values(0).unique()
ss.studies = ss.studies.loc[res_idx]
ss.refresh_dim_data()

rg = ss.dimensions['result_groups'].data
sp = ss.dimensions['sponsors'].data

Syncing the temp table temp_cur_studies in 80 chunks x 500 records each

Creating index on the temp table
 - Loading dimension sponsors
 -- Loading raw data
 -- Sorting index
 - Loading dimension result_groups
 -- Loading raw data
 -- Sorting index


### The types of results have too much variety to be able to process all at the same time, as a result, I think we should just look at specific diseases that have common measurements such as Cancer to start.

## Loading the stock return data

In [13]:
df_tr = pd.read_pickle("stock_data.p")

# Test 1:

@Desc: test whether stocks that publish non-significant tests underperform others in a n-day period after publishing the study

In [15]:
rg

,id,ctgov_group_code,result_type,title,description
nct_id,,,,,
NCT00000125,6437753,B3,Baseline,Total,Total of all reporting groups
NCT00000125,6437754,B2,Baseline,Treatment,Topical Antiglaucoma Agents: Topical Antiglauc...
NCT00000125,6437755,B1,Baseline,Observation,Observation only
NCT00000125,6437756,P2,Participant Flow,Treatment,Topical ocular hypotensive eye drops.
NCT00000125,6437757,P1,Participant Flow,Observation,Close Observation
...,...,...,...,...,...
NCT04272775,6851607,O1,Outcome,Cohort 1: Ixazomib 4.0 mg,"Ixazomib 4.0 mg, capsules, orally, once, on Da..."
NCT04272775,6851608,O2,Outcome,Cohort 2: Ixazomib 4.0 mg + Lenalidomide and D...,"Ixazomib 4.0 mg, capsules, orally, once, on Da..."
NCT04272775,6851609,O1,Outcome,Cohort 1: Ixazomib 4.0 mg,"Ixazomib 4.0 mg, capsules, orally, once, on Da..."
